##
- train: 학습용
- val: 검증용
> 기존 데이터 셋, 8:2 split (train, val)

- test: 최종 테스트용

In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from pytorch_tabular import TabularModel
from pytorch_tabular.models import (
CategoryEmbeddingModelConfig,
FTTransformerConfig,
TabNetModelConfig,
GANDALFConfig,
)
from pytorch_tabular.config import DataConfig, OptimizerConfig, TrainerConfig
from pytorch_tabular.models.stacking import StackingModelConfig
# from pytorch_tabular.utils import make_mixed_dataset

from sklearn.preprocessing import LabelEncoder, FunctionTransformer, QuantileTransformer, MultiLabelBinarizer

from sklearn.impute import SimpleImputer

from sklearn.model_selection import StratifiedKFold

from sklearn.metrics import roc_auc_score

import random

import preprocessing

from pytorch_lightning.loggers import WandbLogger

In [2]:
data_seed = 1
seed = 333

train_path = f'../../data/custom_train_{data_seed}.csv'
test_path = f'../../data/custom_test_{data_seed}.csv'

# 학습/평가 데이터 로드
train = pd.read_csv(train_path).drop(columns=['ID'])
test = pd.read_csv(test_path).drop(columns=['ID'])

print(train.shape, test.shape)

(205080, 68) (51271, 67)


## preprocessing

In [3]:
train, test = preprocessing.all_process(train, test)

print(train.shape, test.shape)

(205080, 66) (51271, 65)


In [4]:
cat_cols = [col for col in train.columns if pd.api.types.is_categorical_dtype(train[col])]
numeric_cols = [col for col in train.columns if col not in cat_cols and col != '임신 성공 여부']

print(f'수치형 변수: {len(numeric_cols)}개 \n{numeric_cols}')
print(f'범주형 변수: {len(cat_cols)}개 \n{cat_cols}')
print(train.shape, test.shape)

수치형 변수: 57개 
['임신 시도 또는 마지막 임신 경과 연수', '배란 자극 여부', '단일 배아 이식 여부', '착상 전 유전 검사 사용 여부', '착상 전 유전 진단 사용 여부', '남성 주 불임 원인', '남성 부 불임 원인', '여성 주 불임 원인', '여성 부 불임 원인', '부부 주 불임 원인', '부부 부 불임 원인', '불명확 불임 원인', '불임 원인 - 난관 질환', '불임 원인 - 남성 요인', '불임 원인 - 배란 장애', '불임 원인 - 자궁경부 문제', '불임 원인 - 자궁내막증', '불임 원인 - 정자 농도', '불임 원인 - 정자 운동성', '불임 원인 - 정자 형태', '클리닉 내 총 시술 횟수', 'IVF 시술 횟수', 'DI 시술 횟수', '총 임신 횟수', 'IVF 임신 횟수', 'DI 임신 횟수', '총 출산 횟수', 'IVF 출산 횟수', 'DI 출산 횟수', '총 생성 배아 수', '미세주입된 난자 수', '미세주입에서 생성된 배아 수', '이식된 배아 수', '미세주입 배아 이식 수', '저장된 배아 수', '미세주입 후 저장된 배아 수', '해동된 배아 수', '해동 난자 수', '수집된 신선 난자 수', '저장된 신선 난자 수', '혼합된 난자 수', '파트너 정자와 혼합된 난자 수', '기증자 정자와 혼합된 난자 수', '동결 배아 사용 여부', '신선 배아 사용 여부', '기증 배아 사용 여부', '대리모 여부', 'PGD 시술 여부', 'PGS 시술 여부', '난자 혼합 경과일', '배아 이식 경과일', '배아 해동 경과일', '시술_임신', '배아생성이유_기증용', '배아생성이유_난자 저장용', '배아생성이유_배아 저장용', '배아생성이유_현재 시술용']
범주형 변수: 8개 
['시술 시기 코드', '시술 당시 나이', '배란 유도 유형', '난자 출처', '정자 출처', '난자 기증자 나이', '정자 기증자 나이', '시술유형_통합']
(205080, 66) (51271, 65)


## config
- continuous_cols 기본 설정 뺴기 @@@@@@@@@@@

In [ ]:
## 기본 학습 관련 config
data_config = DataConfig(
    target=["임신 성공 여부"],
    continuous_cols=numeric_cols,
    categorical_cols=cat_cols,
    normalize_continuous_features=False,     # 정규화 기본 설정 False로 수정
)
trainer_config = TrainerConfig(
    batch_size=4096,
    max_epochs=20,
    early_stopping="valid_loss",     
    early_stopping_mode="min",
    early_stopping_patience=3,
    # checkpoints="valid_loss",        
    load_best=True, 
    auto_lr_find=False,
    seed=333
    
)
optimizer_config = OptimizerConfig()  # default: Adam, 1e-3

## stacking 할 모델들 config
model_config_1 = CategoryEmbeddingModelConfig(
    task="classification",
    layers="128-64-32",
    activation="ReLU",
    learning_rate=1e-3,
    seed=seed
)
model_config_2 = FTTransformerConfig(
    task="classification",
    input_embed_dim=32,
    num_attn_blocks=2,
    num_heads=4,
    learning_rate=1e-3,
    seed=seed
)
# model_config_3 = TabNetModelConfig(
#     task="classification",
#     n_d=16,
#     n_a=16,
#     n_steps=5,
#     learning_rate=1e-3,
#     seed=seed
# )
model_config_4 = GANDALFConfig(
    task="classification",
    gflu_stages=6,
    gflu_dropout=0.1,
    gflu_feature_init_sparsity=0.3,  # 각 GFLU 스테이지에서 처음에 선택할 feature의 비율
    learnable_sparsity=True,  # GFLU에서 선택할 feature의 sparsity 비율을 학습 중에 업데이트할지 여부
    embedding_dropout=0.05,
    batch_norm_continuous_input=False,  # 연속형 정규화 안함
    learning_rate=1e-3,
    seed=seed,    
)

## SoftVoting

In [6]:
# ROC AUC
def _roc_auc_scoer(y_true, y_pred):
    return roc_auc_score(y_true, y_pred['임신 성공 여부_1_probability'])

# 모델 설정 리스트 (각 모델의 config)
model_configs = [model_config_1, model_config_2, model_config_4]

# 각 모델의 예측 확률을 저장할 리스트와 개별 AUC 기록 리스트
predictions = []
individual_auc = []

test_preds = []

seed = 333
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)

for i, config in enumerate(model_configs):
    # 각 모델 독립적으로 초기화
    main_model = TabularModel(
        data_config=data_config,
        model_config=config,
        optimizer_config=optimizer_config,
        trainer_config=trainer_config
    )
    
    # 학습/평가 데이터 로드
    train = pd.read_csv(train_path).drop(columns=['ID'])
    test = pd.read_csv(test_path).drop(columns=['ID'])
    
    # 각 모델 폴드별 roc 확인용
    roc_metrics = []
    
    # StratifiedKFold
    for fold, (train_idx, val_idx) in enumerate(skf.split(train, train['임신 성공 여부'])):
        
        # 현재 fold의 train/validation 데이터 분할
        train_fold = train.iloc[train_idx].copy().reset_index(drop=True)
        val_fold = train.iloc[val_idx].copy().reset_index(drop=True)    
        
        train2_fold = train_fold.copy()
        test_fold = test.copy() 
        
        # preprocessing
        train_fold, val_fold = preprocessing.all_process(train_fold, val_fold)
        train2_fold, test_fold = preprocessing.all_process(train2_fold, test_fold)
        
        # 첫 fold일 때 datamodule과 모델 초기화, 이후 fold에서는 copy로 재사용
        if fold == 0:
            datamodule = main_model.prepare_dataloader(train=train_fold, validation=val_fold, seed=seed)
            model = main_model.prepare_model(datamodule)
        else:
            datamodule = datamodule.copy(train=train_fold, validation=val_fold)
        
        # Train
        main_model.train(model, datamodule)
        
        # Test
        pred_test = main_model.predict(test_fold)
        test_preds.append(pred_test)
        
        # Val
        pred_df = main_model.predict(val_fold)
        
        # Evaluation
        fold_roc = _roc_auc_scoer(val_fold['임신 성공 여부'], pred_df)
        roc_metrics.append(fold_roc)

        print(f'model: {config.__class__.__name__} | Fold{fold+1} | ROC AUC:{fold_roc:.8f}')
        
        # 모델 가중치 초기화
        main_model.model.reset_weights()
    
    # 각 모델별 5개 폴드 평가지표(AUC) 평균
    average_roc_auc = np.mean(roc_metrics, axis=0)
    print('='*30)
    print(f'model: {config.__class__.__name__} | Average AUC: {average_roc_auc:.8f}')
    print('='*30)
    
# 최종 예측 확률
final_test_preds = np.mean(test_preds, axis=0)
        

2025-04-01 08:39:20,110 - {pytorch_tabular.tabular_model:147} - INFO - Experiment Tracking is turned off

2025-04-01 08:39:31,350 - {pytorch_tabular.tabular_model:549} - INFO - Preparing the DataLoaders

2025-04-01 08:39:31,570 - {pytorch_tabular.tabular_datamodule:527} - INFO - Setting up the datamodule for          
classification task

2025-04-01 08:39:32,159 - {pytorch_tabular.tabular_model:600} - INFO - Preparing the Model: CategoryEmbeddingModel

2025-04-01 08:39:32,430 - {pytorch_tabular.tabular_model:343} - INFO - Preparing the Trainer

...

Trainable params: 256 K                                                                                            
Non-trainable params: 0                                                                                            
Total params: 256 K                                                                                                
Total estimated model params size (MB): 1                                                                          
Modules in train mode: 34                                                                                          
Modules in eval mode: 0

Output()

2025-04-01 09:20:50,550 - {pytorch_tabular.tabular_model:692} - INFO - Training the model completed

2025-04-01 09:20:50,553 - {pytorch_tabular.tabular_model:1533} - INFO - Loading the best model

model: GANDALFConfig | Fold5 | ROC AUC:0.73613726
model: GANDALFConfig | Average AUC: 0.73521685


In [7]:
pred_test

,임신 성공 여부_0_probability,임신 성공 여부_1_probability,임신 성공 여부_prediction
0,0.802614,0.197386,0
1,0.781448,0.218552,0
2,0.998149,0.001851,0
3,0.791299,0.208701,0
4,0.589671,0.410329,0
...,...,...,...
51266,0.998547,0.001453,0
51267,0.764433,0.235567,0
51268,0.894889,0.105111,0
51269,0.999059,0.000941,0


## submission

In [8]:
final_test_preds

array([[0.82503584, 0.17496418, 0.        ],
       [0.78839782, 0.21160218, 0.        ],
       [0.99728158, 0.00271842, 0.        ],
       ...,
       [0.90703705, 0.09296295, 0.        ],
       [0.99881589, 0.00118411, 0.        ],
       [0.99812315, 0.00187686, 0.        ]])

In [9]:
tmp_submission = pd.DataFrame({f'stacking_{data_seed}': final_test_preds[:,1]})
tmp_submission

,stacking_1
0,0.174964
1,0.211602
2,0.002718
3,0.230700
4,0.384932
...,...
51266,0.001802
51267,0.226428
51268,0.092963
51269,0.001184


In [10]:
import sys
sys.path.append("../../")
from cal_auc import calculate_auc

score = calculate_auc(tmp_submission, seed=data_seed)
print(f'[Seed: {data_seed}]: {score}')

[Seed: 1]: 0.7375756927798183
